In [101]:
import edgar as ed
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd
edregex = re.compile(r"enddate", re.IGNORECASE) #search for enddate tags regardless of case cause edgar is inconsistent
sdregex = re.compile(r"startdate", re.IGNORECASE) #search for startdate tags regardless of case cause edgar is inconsistent
tagtrim = re.compile(r"<(?:endDate|startDate)>(.*?)</(?:endDate|startDate)>", re.IGNORECASE)
gaapre = re.compile(r"us-gaap:./*", re.IGNORECASE)

In [2]:
cik = "858877" #csco
recents = ed.get_recent_filings(cik, '10-Q') #quarterlies


[EDGAR :: INFO] 
Getting filings from URL: https://data.sec.gov/submissions/CIK0000858877.json
[EDGAR :: DEBUG] Requesting forms from: https://www.sec.gov/Archives/edgar/data/858877/000085887725000007
[EDGAR :: DEBUG] Requesting forms from: https://www.sec.gov/Archives/edgar/data/858877/000085887724000030
[EDGAR :: DEBUG] Requesting forms from: https://www.sec.gov/Archives/edgar/data/858877/000085887724000007
[EDGAR :: DEBUG] Requesting forms from: https://www.sec.gov/Archives/edgar/data/858877/000085887724000003
[EDGAR :: DEBUG] Requesting forms from: https://www.sec.gov/Archives/edgar/data/858877/000085887723000031
[EDGAR :: DEBUG] Requesting forms from: https://www.sec.gov/Archives/edgar/data/858877/000085887723000013
[EDGAR :: DEBUG] Requesting forms from: https://www.sec.gov/Archives/edgar/data/858877/000085887723000005
[EDGAR :: DEBUG] Requesting forms from: https://www.sec.gov/Archives/edgar/data/858877/000085887722000025
[EDGAR :: DEBUG] Requesting forms from: https://www.sec.g

In [14]:
soup = BeautifulSoup(recents[0].text, features="xml")

In [40]:
cons = soup.find_all('context')
cids = {}
for c in cons:
    enddate = c.find(edregex)
    if enddate:
        cids[c.attrs['id']] = {'enddate': enddate.text, 'startdate': c.find(sdregex).text}
        

In [62]:
def find_by_id(id, tagName=None):
    if tagName is None:
        els = soup.find_all(attrs={'contextRef': id})
    else:
        els = soup.find_all(tagName, attrs={'contextRef': id})
    return els

In [59]:
gaps = soup.find_all(gaapre)

In [71]:
cur = find_by_id('c-1', gaapre)
for c in cur:
    id = c.attrs['contextRef']
    #print(cids[id]['enddate'] + " - " + c.name)
    print(f"\n\n{c.name} :: {c.attrs} :: {c.text}")



RevenueFromContractWithCustomerExcludingAssessedTax :: {'contextRef': 'c-1', 'decimals': '-6', 'id': 'f-124', 'unitRef': 'usd'} :: 27832000000


CostOfGoodsAndServicesSold :: {'contextRef': 'c-1', 'decimals': '-6', 'id': 'f-136', 'unitRef': 'usd'} :: 9600000000


GrossProfit :: {'contextRef': 'c-1', 'decimals': '-6', 'id': 'f-140', 'unitRef': 'usd'} :: 18232000000


ResearchAndDevelopmentExpense :: {'contextRef': 'c-1', 'decimals': '-6', 'id': 'f-144', 'unitRef': 'usd'} :: 4585000000


SellingAndMarketingExpense :: {'contextRef': 'c-1', 'decimals': '-6', 'id': 'f-148', 'unitRef': 'usd'} :: 5424000000


GeneralAndAdministrativeExpense :: {'contextRef': 'c-1', 'decimals': '-6', 'id': 'f-152', 'unitRef': 'usd'} :: 1547000000


AmortizationOfIntangibleAssets :: {'contextRef': 'c-1', 'decimals': '-6', 'id': 'f-156', 'unitRef': 'usd'} :: 530000000


OperatingExpenses :: {'contextRef': 'c-1', 'decimals': '-6', 'id': 'f-164', 'unitRef': 'usd'} :: 12761000000


OperatingIncomeLoss :: {'contex

In [87]:
sids = sorted(cids.keys(), key=lambda c: cids[c]['enddate'])
for c in sids:
    print(f"{c} :: {cids[c]['startdate']} - {cids[c]['enddate']}")

c-419 :: 2016-08-08 - 2016-08-08
c-411 :: 2018-02-13 - 2018-02-13
c-417 :: 2019-06-12 - 2019-06-12
c-414 :: 2020-04-01 - 2022-02-28
c-415 :: 2020-04-01 - 2022-02-28
c-418 :: 2021-09-28 - 2022-05-24
c-420 :: 2016-08-09 - 2023-07-29
c-425 :: 2023-07-30 - 2023-10-28
c-416 :: 2021-01-01 - 2023-12-31
c-6 :: 2023-10-29 - 2024-01-27
c-8 :: 2023-07-30 - 2024-01-27
c-10 :: 2023-10-29 - 2024-01-27
c-12 :: 2023-07-30 - 2024-01-27
c-14 :: 2023-10-29 - 2024-01-27
c-15 :: 2023-07-30 - 2024-01-27
c-44 :: 2023-10-29 - 2024-01-27
c-45 :: 2023-10-29 - 2024-01-27
c-46 :: 2023-10-29 - 2024-01-27
c-47 :: 2023-10-29 - 2024-01-27
c-56 :: 2023-07-30 - 2024-01-27
c-57 :: 2023-07-30 - 2024-01-27
c-58 :: 2023-07-30 - 2024-01-27
c-59 :: 2023-07-30 - 2024-01-27
c-61 :: 2023-10-29 - 2024-01-27
c-63 :: 2023-07-30 - 2024-01-27
c-65 :: 2023-10-29 - 2024-01-27
c-67 :: 2023-07-30 - 2024-01-27
c-69 :: 2023-10-29 - 2024-01-27
c-71 :: 2023-07-30 - 2024-01-27
c-73 :: 2023-10-29 - 2024-01-27
c-75 :: 2023-07-30 - 2024-01-27
c

In [95]:
cids['c-1']

{'enddate': '2025-01-25', 'startdate': '2024-07-28'}

In [111]:
pend = "10/2024" #period end as pulled from my db
#pids = []
#pend = datetime.strptime(pend, "%m/%Y").date()
#print("Period end = ", pend)

pd_period = pd.Period(pend, freq="M")
print(pd_period)
for c in cids:
    cend = datetime.strptime(cids[c]['enddate'], "%Y-%m-%d").date()
    #print(f"\n{cend}")
    cend = pd.Period(cids[c]['enddate'], freq="M")
    if pd_period == cend:
        print(c)

2024-10
c-422


In [124]:
def find_id_by_pend(pend, cids): # period end passed in in "10/2024" format - direct from db period_end column
    tar_period = pd.Period(pend, freq="M")

    for c in cids:
        c_end = pd.Period(cids[c]['enddate'], freq="M")
        if c_end == tar_period:
            print(f"\n[{c}] Match found!", cids[c])
            c_start = pd.Period(cids[c]['startdate'], freq="M")
            period_len = (c_end - c_start).n
            print("Period length is: ", period_len)
            print(type(period_len))
            if period_len == 3:
                print(f"******************** {c} is id for a valid quarter! ********************")

In [125]:
find_id_by_pend("10/2024", cids)


[c-422] Match found! {'enddate': '2024-10-26', 'startdate': '2024-07-28'}
Period length is:  3
<class 'int'>
******************** c-422 is id for a valid quarter! ********************
